In [6]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('/content/diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
X = df.iloc[:,:-1]
y= df.iloc[:,-1:]

In [6]:
y

,Outcome
0,1
1,0
2,1
3,0
4,1
...,...
763,0
764,0
765,0
766,1


In [6]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=1)

In [7]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [10]:
model = Sequential()

model.add(Dense(32, activation='relu', input_dim=8)) #imput and hidden layer
model.add(Dense(1, activation='sigmoid')) #output layer

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(X_train,y_train, batch_size=32, epochs=100, validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 15ms/step - loss: 7.7265 - accuracy: 0.4007 - val_loss: 3.5743 - val_accuracy: 0.5325
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 3.5563 - accuracy: 0.5000 - val_loss: 2.7109 - val_accuracy: 0.5195
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 2.5447 - accuracy: 0.5375 - val_loss: 2.1333 - val_accuracy: 0.5390
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 1.8704 - accuracy: 0.5309 - val_loss: 1.7275 - val_accuracy: 0.5714
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 1.5170 - accuracy: 0.5635 - val_loss: 1.6344 - val_accuracy: 0.5260
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 1.3298 - accuracy: 0.5912 - val_loss: 1.4135 - val_accuracy: 0.5455
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 1.1882 - accuracy: 0.6010 - val_loss: 1.2830 - val_accuracy: 0.5779
Epoch 8/100


In [12]:
#1. How to select appropriate optimizer
#2. No. of nodes in a layer
#3. How to select no. of layers
#4. All in one model

In [8]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.5 MB/s eta 0:00:00


#1. How to select appropriate optimizer

In [10]:
import keras_tuner as kt

In [15]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(32, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  optimizer = hp.Choice('optimizer', values= ['adam','rmsprop','sgd','adadelta'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [16]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=6)

In [17]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.701298713684082

Best val_accuracy So Far: 0.701298713684082
Total elapsed time: 00h 00m 14s


In [18]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'sgd'}

In [19]:
model = tuner.get_best_models(num_models=1)[0]

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
model.fit(X_train,y_train, batch_size=32,epochs=100, initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.6316 - accuracy: 0.6889 - val_loss: 0.7630 - val_accuracy: 0.6494
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6510 - accuracy: 0.6808 - val_loss: 0.8435 - val_accuracy: 0.5584
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6607 - accuracy: 0.6824 - val_loss: 0.6694 - val_accuracy: 0.6688
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6252 - accuracy: 0.6954 - val_loss: 0.6026 - val_accuracy: 0.6623
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5926 - accuracy: 0.6922 - val_loss: 0.6529 - val_accuracy: 0.6169
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5966 - accuracy: 0.6938 - val_loss: 0.6535 - val_accuracy: 0.6558
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6103 - accuracy: 0.6808 - val_loss: 0.7891 - val_accuracy: 0.6429
Epoch 14

#2. No. of nodes in a layer

In [22]:
def build_model(hp):

  model = Sequential()

  units = hp.Int("units", min_value=8, max_value=128) #lower_bound, upper_bound, steps 8+8+8+8
  model.add(Dense(units=units, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [23]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='mydir', project_name='Tunning')

In [24]:
tuner.search(X_train,y_train,epochs=5, validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.6753246784210205

Best val_accuracy So Far: 0.6753246784210205
Total elapsed time: 00h 00m 08s


In [25]:
tuner.get_best_hyperparameters()[0].values

{'units': 105}

In [26]:
model = tuner.get_best_models(num_models=1)[0]

In [27]:
model.fit(X_train,y_train,batch_size=32,epochs=100, initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 11ms/step - loss: 0.9555 - accuracy: 0.6336 - val_loss: 1.5901 - val_accuracy: 0.6299
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.8581 - accuracy: 0.6564 - val_loss: 1.8394 - val_accuracy: 0.3961
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.9260 - accuracy: 0.6270 - val_loss: 0.8966 - val_accuracy: 0.6558
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.9726 - accuracy: 0.6238 - val_loss: 2.1068 - val_accuracy: 0.3701
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.9055 - accuracy: 0.6336 - val_loss: 1.3229 - val_accuracy: 0.6299
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.8248 - accuracy: 0.6710 - val_loss: 1.0385 - val_accuracy: 0.6299
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.9010 - accuracy: 0.6596 - val_loss: 2.4232 - val_accuracy: 0.6429
Epoch 14

#3. How to select no. of layers

In [61]:
# Install Keras Tuner if not already installed
# pip install keras-tuner

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

# Define the model-building function
def build_model(hp):
    model = keras.Sequential()

    # Tune the number of layers
    num_layers = hp.Int('num_layers', min_value=1, max_value=5, step=1)
    for i in range(num_layers):
        # Tune the number of units in each layer
        units = hp.Int(f'units_{i}', min_value=8, max_value=128, step=8)
        model.add(layers.Dense(units, activation='relu'))

    model.add(layers.Dense(1, activation='sigmoid'))  # Output layer

    # Compile the model
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

<ipython-input-61-a098e32d0080>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [62]:
# Instantiate the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Number of hyperparameter combinations to try
    directory='my_tuner_dir',  # Directory to store the tuning results
    project_name='my_ann_tuning'  # Name of the tuning project
)

In [63]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.6558441519737244

Best val_accuracy So Far: 0.6688311696052551
Total elapsed time: 00h 00m 12s


In [64]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3, 'units_0': 56, 'units_1': 96, 'units_2': 40, 'units_3': 88}

In [67]:
model = tuner.get_best_models(num_models=1)[0]

In [69]:
model.fit(X_train,y_train,epochs=100, initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 2s 21ms/step - loss: 0.7528 - accuracy: 0.6417 - val_loss: 0.7622 - val_accuracy: 0.6299
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.7259 - accuracy: 0.6596 - val_loss: 0.8861 - val_accuracy: 0.6039
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.8127 - accuracy: 0.6515 - val_loss: 0.8342 - val_accuracy: 0.6494
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6474 - accuracy: 0.6906 - val_loss: 0.8373 - val_accuracy: 0.6234
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6703 - accuracy: 0.6694 - val_loss: 0.7857 - val_accuracy: 0.6818
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5807 - accuracy: 0.7003 - val_loss: 0.6807 - val_accuracy: 0.6948
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6280 - accuracy: 0.6889 - val_loss: 0.7756 - val_accuracy: 0.6104
Epoch 14

#4. All in one model

In [35]:
def build_model(hp):

  model = Sequential()

  counter = 0


  for i in range(hp.Int('num_lays', min_value=1, max_value=10)):

    if counter == 0:
      model.add(
          Dense(hp.Int('units'+str(i), min_value=8, max_value=128,step=8),
                       activation= hp.Choice('activation'+str(i), values=['relu','tanh','sigmoid']),input_dim=8
          )
      )
    else:
       model.add(
          Dense(hp.Int('units'+str(i), min_value=8, max_value=128,step=8),
                       activation= hp.Choice('activation'+str(i), values=['relu','tanh','sigmoid'])
          )
      )
    counter+=1

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop','adam','sgd','nadam','adadelta']),
                loss='binary_crossentropy', metrics=['accuracy']
                )
  return model


In [36]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='all_in_one')

Reloading Tuner from mydir/all_in_one/tuner0.json


In [37]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [38]:
tuner.get_best_hyperparameters()[0].values

{'num_lays': 6,
 'units0': 128,
 'activation0': 'tanh',
 'optimizer': 'adam',
 'units1': 32,
 'activation1': 'relu',
 'units2': 24,
 'activation2': 'relu',
 'units3': 56,
 'activation3': 'relu',
 'units4': 88,
 'activation4': 'tanh',
 'units5': 112,
 'activation5': 'sigmoid',
 'units6': 64,
 'activation6': 'relu',
 'units7': 80,
 'activation7': 'relu'}

In [39]:
model = tuner.get_best_models(num_models=1)[0]

In [40]:
model.fit(X_train,y_train,epochs=100, initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 2s 21ms/step - loss: 0.5553 - accuracy: 0.7248 - val_loss: 0.6459 - val_accuracy: 0.6948
Epoch 8/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5470 - accuracy: 0.6954 - val_loss: 0.6928 - val_accuracy: 0.6688
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5620 - accuracy: 0.6840 - val_loss: 0.6515 - val_accuracy: 0.6818
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5329 - accuracy: 0.7182 - val_loss: 0.6786 - val_accuracy: 0.6623
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5433 - accuracy: 0.7345 - val_loss: 0.6342 - val_accuracy: 0.7013
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5517 - accuracy: 0.7003 - val_loss: 0.6380 - val_accuracy: 0.6818
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5386 - accuracy: 0.7003 - val_loss: 0.6417 - val_accuracy: 0.6753
Epoch 14

#All in one with dropout

In [29]:
def build_model(hp):

  model = Sequential()

  counter = 0


  for i in range(hp.Int('num_lays', min_value=1, max_value=10)):

    if counter == 0:
      model.add(
          Dense(hp.Int('units'+str(i), min_value=8, max_value=128,step=8),
                       activation= hp.Choice('activation'+str(i), values=['relu','tanh','sigmoid']), input_dim=8))
      model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
       model.add(
          Dense(hp.Int('units'+str(i), min_value=8, max_value=128,step=8),
                       activation= hp.Choice('activation'+str(i), values=['relu','tanh','sigmoid'])))
       model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter+=1

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop','adam','sgd','nadam','adadelta']),
                loss='binary_crossentropy', metrics=['accuracy']
                )
  return model


In [30]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='all_in_one_dropout')

In [31]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 10s


In [32]:
tuner.get_best_hyperparameters()[0].values

{'num_lays': 8,
 'units0': 56,
 'activation0': 'tanh',
 'dropout0': 0.6,
 'optimizer': 'sgd',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'dropout4': 0.1,
 'units5': 8,
 'activation5': 'relu',
 'dropout5': 0.1,
 'units6': 8,
 'activation6': 'relu',
 'dropout6': 0.1,
 'units7': 8,
 'activation7': 'relu',
 'dropout7': 0.1}

In [33]:
model = tuner.get_best_models(num_models=1)[0]

In [34]:
model.fit(X_train,y_train,epochs=100, initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 2s 17ms/step - loss: 0.6773 - accuracy: 0.6238 - val_loss: 0.6734 - val_accuracy: 0.6429
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6731 - accuracy: 0.6401 - val_loss: 0.6699 - val_accuracy: 0.6429
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6678 - accuracy: 0.6450 - val_loss: 0.6661 - val_accuracy: 0.6429
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6652 - accuracy: 0.6564 - val_loss: 0.6635 - val_accuracy: 0.6429
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6618 - accuracy: 0.6450 - val_loss: 0.6610 - val_accuracy: 0.6429
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6551 - accuracy: 0.6531 - val_loss: 0.6590 - val_accuracy: 0.6429
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6585 - accuracy: 0.6482 - val_loss: 0.6567 - val_accuracy: 0.6429
Epoch 14

#all in one my version

In [2]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.6 MB/s eta 0:00:00


In [36]:
# Install Keras Tuner if not already installed
# pip install keras-tuner
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner
from tensorflow import keras
from keras import layers
from keras import Sequential
from keras.layers import Dense, Dropout
from kerastuner.tuners import RandomSearch


# Define the model-building function
def build_model(hp):
    model = keras.Sequential()

    # Tune the choice of optimizer
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])

    # Tune the number of layers
    num_layers = hp.Int('num_layers', min_value=1, max_value=10, step=1)
    counter=0
    for i in range(num_layers):

      if counter == 0:
        # Tune the choice of activation function
        activation = hp.Choice(f'activation_{i}', values=['relu', 'sigmoid', 'tanh'])

        # Tune the number of units in each layer
        units = hp.Int(f'units_{i}', min_value=8, max_value=128, step=8)

        model.add(Dense(units, activation=activation, input_dim=8))

        # Tune dropout rate for each layer
        dropout_rate = hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1)
        model.add(Dropout(rate=dropout_rate))

      else:
        # Tune the choice of activation function
        activation = hp.Choice(f'activation_{i}', values=['relu', 'sigmoid', 'tanh'])

        # Tune the number of units in each layer
        units = hp.Int(f'units_{i}', min_value=8, max_value=128, step=8)

        model.add(Dense(units, activation=activation))

        # Tune dropout rate for each layer
        dropout_rate = hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1)
        model.add(Dropout(rate=dropout_rate))
      counter +=1


    model.add(layers.Dense(1, activation='sigmoid'))  # Output layer

    # Tune the learning rate
    lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model


In [37]:
# Instantiate the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Number of hyperparameter combinations to try
    directory='my_tuner_dir',  # Directory to store the tuning results
    project_name='my_ann_tuning3'  # Name of the tuning project
)

In [38]:
df = pd.read_csv('/content/diabetes.csv')
X = df.iloc[:,:-1]
y= df.iloc[:,-1:]
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=1)

In [39]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.6298701167106628

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 18s


In [16]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop',
 'num_layers': 10,
 'activation_0': 'sigmoid',
 'units_0': 56,
 'dropout_0': 0.2,
 'learning_rate': 0.01,
 'activation_1': 'relu',
 'units_1': 8,
 'dropout_1': 0.1,
 'activation_2': 'relu',
 'units_2': 8,
 'dropout_2': 0.1,
 'activation_3': 'relu',
 'units_3': 8,
 'dropout_3': 0.1,
 'activation_4': 'relu',
 'units_4': 8,
 'dropout_4': 0.1,
 'activation_5': 'relu',
 'units_5': 8,
 'dropout_5': 0.1,
 'activation_6': 'relu',
 'units_6': 8,
 'dropout_6': 0.1,
 'activation_7': 'relu',
 'units_7': 8,
 'dropout_7': 0.1,
 'activation_8': 'relu',
 'units_8': 8,
 'dropout_8': 0.1,
 'activation_9': 'relu',
 'units_9': 8,
 'dropout_9': 0.1}

In [17]:
model = tuner.get_best_models(num_models=1)[0]

In [18]:
model.fit(X_train,y_train,epochs=100, initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 2s 18ms/step - loss: 0.6794 - accuracy: 0.6482 - val_loss: 0.6763 - val_accuracy: 0.6429
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6718 - accuracy: 0.6547 - val_loss: 0.6702 - val_accuracy: 0.6429
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6644 - accuracy: 0.6482 - val_loss: 0.6639 - val_accuracy: 0.6429
Epoch 10/100
20/20 [==============================] - 0s 7ms/step - loss: 0.6604 - accuracy: 0.6515 - val_loss: 0.6599 - val_accuracy: 0.6429
Epoch 11/100
20/20 [==============================] - 0s 7ms/step - loss: 0.6530 - accuracy: 0.6531 - val_loss: 0.6569 - val_accuracy: 0.6429
Epoch 12/100
20/20 [==============================] - 0s 7ms/step - loss: 0.6500 - accuracy: 0.6531 - val_loss: 0.6544 - val_accuracy: 0.6429
Epoch 13/100
20/20 [==============================] - 0s 7ms/step - loss: 0.6490 - accuracy: 0.6531 - val_loss: 0.6538 - val_accuracy: 0.6429
Epoch 14

In [19]:
# Install Keras Tuner if not already installed
# pip install keras-tuner

import tensorflow as tf
from tensorflow import keras
from keras import layers
from kerastuner.tuners import Hyperband

# Define the model-building function
def build_model(hp):
    model = keras.Sequential()

    # Tune the number of layers
    num_layers = hp.Int('num_layers', min_value=1, max_value=5, step=1)
    for i in range(num_layers):
        # Tune the number of units in each layer
        units = hp.Int(f'units_{i}', min_value=8, max_value=128, step=8)
        model.add(layers.Dense(units, activation='relu'))

        # Tune dropout rate for each layer
        dropout_rate = hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1)
        model.add(layers.Dropout(rate=dropout_rate))

    model.add(layers.Dense(1, activation='sigmoid'))  # Output layer

    # Tune the learning rate
    lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model



In [21]:
# Instantiate the Hyperband tuner
tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,  # Maximum number of training epochs for each configuration
    factor=3,       # Reduction factor for the number of configurations in each bracket
    directory='my_tuner_dir',  # Directory to store the tuning results
    project_name='my_ann_tuning1'  # Name of the tuning project
)

In [22]:
df = pd.read_csv('/content/diabetes.csv')
X = df.iloc[:,:-1]
y= df.iloc[:,-1:]
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=1)

In [23]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 30 Complete [00h 00m 05s]
val_accuracy: 0.6818181872367859

Best val_accuracy So Far: 0.6818181872367859
Total elapsed time: 00h 01m 17s


In [24]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2,
 'units_0': 120,
 'dropout_0': 0.0,
 'learning_rate': 0.001,
 'units_1': 64,
 'dropout_1': 0.4,
 'units_2': 48,
 'dropout_2': 0.1,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 4,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0012',
 'units_3': 72,
 'dropout_3': 0.0,
 'units_4': 88,
 'dropout_4': 0.2}

In [25]:
model = tuner.get_best_models(num_models=1)[0]

In [26]:
model.fit(X_train,y_train,epochs=100, initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 2s 19ms/step - loss: 0.6940 - accuracy: 0.6498 - val_loss: 0.6721 - val_accuracy: 0.6818
Epoch 8/100
20/20 [==============================] - 0s 7ms/step - loss: 0.6646 - accuracy: 0.6564 - val_loss: 0.6950 - val_accuracy: 0.6299
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6400 - accuracy: 0.6417 - val_loss: 0.6992 - val_accuracy: 0.6429
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6186 - accuracy: 0.6873 - val_loss: 0.7005 - val_accuracy: 0.6753
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6163 - accuracy: 0.6906 - val_loss: 0.7063 - val_accuracy: 0.6623
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6414 - accuracy: 0.6938 - val_loss: 0.7167 - val_accuracy: 0.6753
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6172 - accuracy: 0.6743 - val_loss: 0.7366 - val_accuracy: 0.6688
Epoch 14